In [1]:
# Something about taking permutations is super slow. what is it?

%load_ext autoreload
%autoreload 2
%matplotlib inline

from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
import scanpy as sc
import pickle

from interaction_score_def import interaction_test_base, _process_permutation, permute_labels
from interaction_util import group_cells
import itertools

In [2]:
from numba import jit, prange

In [3]:
!ls /home/ingn/proj/pembroRT/data/*.h5ad

/home/ingn/proj/pembroRT/data/epithelial_v4_rnd2_filtered.h5ad
/home/ingn/proj/pembroRT/data/PembroRT_v4_rnd2.clust.h5ad
/home/ingn/proj/pembroRT/data/PembroRT_v4_rnd2.receptor_scores.h5ad


In [7]:
#https://github.com/numba/numba/issues/1269#issuecomment-702665837
@jit(nopython=True)
def apply_along_axis_0(func1d, arr):
    """Like calling func1d(arr, axis=0)"""
    if arr.size == 0:
        raise RuntimeError("Must have arr.size > 0")
    ndim = arr.ndim
    if ndim == 0:
        raise RuntimeError("Must have ndim > 0")
    elif 1 == ndim:
        return func1d(arr)
    else:
        result_shape = arr.shape[1:]
        out = np.empty(result_shape, arr.dtype)
        _apply_along_axis_0(func1d, arr, out)
        return out


@jit(nopython=True)
def _apply_along_axis_0(func1d, arr, out):
    """Like calling func1d(arr, axis=0, out=out). Require arr to be 2d or bigger."""
    ndim = arr.ndim
    if ndim < 2:
        raise RuntimeError("_apply_along_axis_0 requires 2d array or bigger")
    elif ndim == 2:  # 2-dimensional case
        for i in range(len(out)):
            out[i] = func1d(arr[:, i])
    else:  # higher dimensional case
        for i, out_slice in enumerate(out):
            _apply_along_axis_0(func1d, arr[:, i], out_slice)


@jit(nopython=True)
def nb_mean_axis_0(arr):
    return apply_along_axis_0(np.mean, arr)


@jit(nopython=True, parallel=True)
def nb_groupby(x, y, uy, min_cells=10):
    xout = np.zeros((len(uy),x.shape[1]), dtype=np.float32)
    for i in prange(len(uy)):
        u = uy[i]
        idx = y==u
        if np.sum(idx) < min_cells:
            continue
        #xout[i,:] = np.mean(x[idx,:], axis=0)
        xout[i,:] = nb_mean_axis_0(x[idx,:])
    return xout

In [5]:
size = 600000
n_groups = 40
n_containers = 120

x = np.expand_dims(np.random.randn(size), 1)
y = np.random.choice(n_groups, size)
c = np.random.choice(n_containers, size)

print(x.shape)
print(y.shape)
print(c.shape)

joined = [f'{i}_{j}' for i,j in zip(y,c)]
uj, ji = np.unique(joined, return_inverse=True)

(600000, 1)
(600000,)
(600000,)


In [8]:
%%timeit -n 5
ret = nb_groupby(x, ji, np.arange(ji.max()), min_cells=10)

224 ms ± 53.9 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [ ]:
# %%timeit
# ret = group_cells(x, ji, u_y=np.arange(ji.max()), agg=lambda x: np.sum(x, axis=0))